# Credentials

Make sure to go through [Setup](/notebooks/credentials/Setup.ipynb) first!

Let's check that the environment variables have been set... We'll just try one:

In [ ]:
%env GPRED_PROJECT_ID

# Google Cloud Storage
Let's see if we can create a bucket with boto (using credentials, project ID, etc. specified in boto config file)...

In [ ]:
import datetime
now = datetime.datetime.now()
BUCKET_NAME = 'test_' + now.strftime("%Y-%m-%d") # lower case letters required, no upper case allowed

import boto
import gcs_oauth2_boto_plugin
project_id = %env GPRED_PROJECT_ID
header_values = {"x-goog-project-id": project_id}
boto.storage_uri(BUCKET_NAME, 'gs').create_bucket(headers=header_values)

Listing existing buckets...

In [ ]:
uri = boto.storage_uri('', 'gs')
# If the default project is defined, call get_all_buckets() without arguments.
for bucket in uri.get_all_buckets(headers=header_values):
  print bucket.name

Upload a file to the new bucket

In [ ]:
import os
os.system("echo 'hello!' > newfile")
filename = 'newfile'
boto.storage_uri(BUCKET_NAME + '/' + filename, 'gs').new_key().set_contents_from_file(open(filename))

See contents of the bucket on the web interface (URL will be outputted below)

In [ ]:
print "https://console.developers.google.com/project/" + project_id + "/storage/browser/" + BUCKET_NAME + "/?authuser=0"

# Google Prediction
## Initialize API wrapper

In [5]:
import googleapiclient.gpred as gpred
oauth_file = %env GPRED_OAUTH_FILE
api = gpred.api(oauth_file)

## Making predictions against a hosted model

Let's use the sample.sentiment hosted model (made publicly available by Google)

In [ ]:
# projectname has to be 414649711441
prediction_request = api.hostedmodels().predict(project='414649711441',
                                     hostedModelName='sample.sentiment',
                                     body={'input': {'csvInstance': ['I hate that stuff is so stupid']}})

result = prediction_request.execute()
# We can print the raw result
print result